一共选取六大类因子中的106个特征值+10个风险因子，使用LightGBM来进行2分类概率预测（分为两种标签值构造）
二分类：
1.按照股票涨跌幅排序，收益前T%标记为1， 其余标记为0
2.按照股票涨跌幅排序，收益前T%标记为1， 后T%标记为-1，中间为0（训练集只用“1，-1”类，删除“0”类，测试集用全部）

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import daolib.dao as dao
import daolib.dso as dso
import util.sectool as sectool
import util.operatetool as optool
import matplotlib.pyplot as plt
import pickle
import time
from tqdm import tqdm
%matplotlib inline

## 读取数据

In [ ]:
stock_price_df = dao.get_security_info('stock_price_info')
all_stock_list = stock_price_df.index.get_level_values(0).drop_duplicates().tolist()
#六大类因子106个
pricevol_factors_list = ['pv_clo_ratemean_1m', 'pv_clo_ratemean_3m', 'pv_clo_ratemean_6m', 'pv_corr_clo_turn_1m',
                        'pv_corr_clo_turn_3m',  'pv_max_ret_1m', 'pv_max_ret_3m', 'pv_cgo_3m', 'pv_iv_capm_hs_3m', 'pv_iv_capm_r2_hs_3m', 'pv_iv_capm_r2_zz_3m',
                        'pv_iv_capm_zz_3m', 'pv_iv_ff_hs_3m', 'pv_iv_ff_r2_hs_3m', 'pv_iv_ff_r2_zz_3m', 'pv_iv_ff_zz_3m',
                        'pv_cgo_1m', 'pv_cgo_6m', 'pv_iv_capm_hs_1m', 'pv_iv_capm_r2_hs_1m', 'pv_iv_capm_r2_zz_1m',
                        'pv_iv_capm_zz_1m', 'pv_iv_ff_hs_1m', 'pv_iv_ff_r2_hs_1m', 'pv_iv_ff_r2_zz_1m', 'pv_iv_ff_zz_1m']

mom_factors_list = ['mm_mom_1m', 'mm_mom_1m_weighted', 'mm_mom_1m_weightedexp',
                    'mm_mom_1y_weightedexp', 'mm_mom_3m', 'mm_mom_3m_weighted', 'mm_mom_3m_weightedexp',
                     'mm_mom_6m_weighted', 'mm_mom_6m_weightedexp']

emotion_factors_list = ['em_cov_volume_1m','em_cov_volume_3m','em_ret_amt_1m',\
                        'em_ret_amt_3m', 'em_ret_amt_6m','em_sl_volume_1m','em_turn_cov_1m',\
                        'em_turn_cov_3m','em_turn_mean_1m', 'em_turn_mean_3m','em_turn_mean_6m',\
                       'em_turn_std_1m', 'em_turn_std_3m','em_turn_std_6m', 'em_turn_stdrate_1m','em_turn_stdrate_3m',\
                       ]

finance_factors_list = ['fn_gross_profit2assets',  'fn_qfa_ebit', 'fn_qfa_ebit2assets',
                        'fn_qfa_ni_from_operating', 'fn_qfa_ni_from_operating2assets', 'fn_qfa_np',
                        'fn_qfa_profit_before_tax2assets', 'fn_qfa_roa', 'fn_qfa_roa_deducted', 'fn_qfa_roe',
                        'fn_qfa_roe_deducted', 'fn_qfa_total_income2assets', 'fn_qfa_tot_income_indus']

growth_factors_list = ['gw_advantage', 'gw_chg_gpmargin_ttm', 'gw_growth_qfa_yoy_gr',
                       'gw_qfa_np_enhance', 'gw_qfa_yoyop', 'gw_qfa_yoy_gr', 'gw_qfa_yoy_or', 'gw_roe_chg',
                       'gw_roe_deducted_chg', 'gw_roe_deducted_enhance', 'gw_roe_deducted_yoy', 'gw_roe_enhance',
                       'gw_roe_yoy', 'gw_trend', 'gw_yoy_qfa_deductedprofit', 'gw_yoy_qfa_np_belongto_parcomsh',
                       'gw_yoy_qfa_roe_deducted']

valuation_factors_list = [  'val_ep_de_qfa', 'val_ep_de_sq',
                          'val_ep_de_ttm', 'val_ep_qfa', 'val_ep_sq', 'val_pge_de',
                          'val_equity_mkt']

estimate_factors_list = ['et_eps_ftm_mom3', 'et_ep_ftm', 'et_ep_ftm_mom1', 'et_ep_ftm_mom3', 'et_ep_ftm_mom6',
                         'et_ep_fy1', 'et_ep_fy2', 'et_gr_ftm_mom3', 'et_np_ftm', 'et_np_ftm_mom3', 'et_np_fy1',
                         'et_np_fy2', 'et_peg_ftm', 'et_roe_ftm']

alpha191_factors_list =['alpha101_013_data','alpha101_015_data','alpha101_016_data','alpha101_026_data',
                        'alpha101_040_data','alpha101_044_data','alpha101_050_data','alpha101_055_data',
                        'alpha101_064_data', 'alpha101_074_data' ,'alpha101_075_data','alpha101_099_data',
                        'alpha191_001_data','alpha191_005_data','alpha191_007_data','alpha191_009_data','alpha191_017_data',
                        'alpha191_022_data','alpha191_030_data','alpha191_031_data','alpha191_032_data','alpha191_036_data',
                        'alpha191_042_data','alpha191_046_data','alpha191_074_data','alpha191_090_data','alpha191_092_data',
                        'alpha191_101_data','alpha191_105_data','alpha191_112_data','alpha191_118_data','alpha191_123_data',
                        'alpha191_124_data','alpha191_139_data','alpha191_173_data']

In [ ]:
#十个风险因子

#股票上市状态(只选择为0的值，不为0的值为异常值)
price_df=dao.get_security_info('stock_price_info')
trade_status_df=price_df.xs('trade_status',level=1)

def risk_factor(alpha_factor_series):
    alpha_factor_series['beta'] = dao.get_style_risk_factor('risk_beta', 'def')
    alpha_factor_series['mv'] = dao.get_style_risk_factor('risk_mv', 'def')
    alpha_factor_series['vol'] = dao.get_style_risk_factor('risk_vol', 'def')
    alpha_factor_series['liq'] = dao.get_style_risk_factor('risk_liq', 'def')
    alpha_factor_series['ey'] = dao.get_style_risk_factor('risk_ey', 'def')
    alpha_factor_series['bp'] = dao.get_style_risk_factor('risk_bp', 'def')
    alpha_factor_series['mom'] = dao.get_style_risk_factor('risk_mom', 'def')
    alpha_factor_series['lev'] = dao.get_style_risk_factor('risk_lev', 'def')
    alpha_factor_series['nlmv'] = dao.get_style_risk_factor('risk_nlmv', 'def')
    alpha_factor_series['gro'] = dao.get_style_risk_factor('risk_gro', 'def')
    alpha_factor_series['trade_status'] = trade_status_df
    return alpha_factor_series

In [ ]:
def read_factor_data(factors_name_list, trade_date_20d_series):
    compound_factor_dict = {}
    alpha_factor_series = pd.Series()
    for compound_name in factors_name_list[:]:
        pro_factor_path = "D:\\Data\\factors\\" + compound_name + '\\'
        #alpha_factor_series = pd.Series()
        factor_list = globals().get(compound_name + '_factors_list')
        for factor_name in factor_list:
#             print('读取 ', factor_name)
            data_df = pd.read_pickle(pro_factor_path + factor_name)
            begin_date = max(data_df.columns[0], trade_date_20d_series[0])
            trade_date_series = trade_date_20d_series[trade_date_20d_series >= begin_date]
            alpha_factor_series.loc[factor_name] = data_df.loc[:, trade_date_series]
    #alpha_factor_df = ftool.factor_class_generate(alpha_factor_series)
        #compound_factor_dict[compound_name] = alpha_factor_df
    return alpha_factor_series

### 获取数据

In [ ]:
start_date = dt.datetime(2007, 1, 1)
end_date = dt.datetime(2018,9,1)
factor_name_list = ['emotion', 'pricevol', 'mom', 'finance', 'valuation', 'alpha191']  #growth

#月频数据
trade_date_m_series = pd.Series( dso.get_trade_date(start_date=start_date, end_date=end_date, period='m') )
month_list = [date for date in trade_date_m_series if dt.datetime(2009,10,1)<date and date < dt.datetime(2018,11,1) ]
month_alpha_factor_series = read_factor_data(factors_name_list=factor_name_list, trade_date_20d_series=trade_date_m_series)
month_alpha_factor_series=risk_factor(month_alpha_factor_series)

#周频数据
trade_date_w_series = pd.Series( dso.get_trade_date(start_date=start_date, end_date=end_date, period='w') )
week_list = [date for date in trade_date_w_series if dt.datetime(2009,10,20)<date and date < dt.datetime(2018,11,1) ]
week_alpha_factor_series = read_factor_data(factors_name_list=factor_name_list, trade_date_20d_series=trade_date_w_series)
week_alpha_factor_series=risk_factor(week_alpha_factor_series)

### 保存数据

In [ ]:
#将上面构造好的数据保存，方便利用
month_alpha_factor_series.to_pickle("./factor_test_data/LightGBM_allfactor2/month_alpha_factor_series")
week_alpha_factor_series.to_pickle("./factor_test_data/LightGBM_allfactor2/week_alpha_factor_series")

### 缺失值处理（一种做处理，一种不作处理，最好不做处理）

In [ ]:
fill_month_alpha_factor_series=nonefill(month_alpha_factor_series,month_list)  #周频数据
fill_week_alpha_factor_series=nonefill(week_alpha_factor_series,week_list)  #周频数据

## 概率预测

In [ ]:
# 月频数据(二分类，按照30%和40%划分标签值)
Mfactor0_lgb4,Mfactor1_lgb4,Mtest_accuracy_lgb4,Mtrain_accuracy_lgb4=predict_probo_class(month_alpha_factor_series,model_lgb,12,month_list,0.4,-1)
Mfactor0_lgb3,Mfactor1_lgb3,Mtest_accuracy_lgb3,Mtrain_accuracy_lgb3=predict_probo_class(month_alpha_factor_series,model_lgb,12,month_list,0.3,-1)

Mfactor0_xgb4,Mfactor1_xgb4,Mtest_accuracy_xgb4,Mtrain_accuracy_xgb4=predict_probo_class(month_alpha_factor_series,model_xgb,12,month_list,0.4,-1)
Mfactor0_xgb3,Mfactor1_xgb3,Mtest_accuracy_xgb3,Mtrain_accuracy_xgb3=predict_probo_class(month_alpha_factor_series,model_xgb,12,month_list,0.3,-1)
                                                                                         
Mfactor0_GBoost4,Mfactor1_GBoost4,Mtest_accuracy_GBoost4,Mtrain_accuracy_GBoost4=predict_probo_class(month_alpha_factor_series,GBoost,12,month_list,0.4,-1)
Mfactor0_GBoost3,Mfactor1_GBoost3,Mtest_accuracy_GBoost3,Mtrain_accuracy_GBoost3=predict_probo_class(month_alpha_factor_series,GBoost,12,month_list,0.3,-1)

Mfactor0_RF4,Mfactor1_RF4,Mtest_accuracy_RF4,Mtrain_accuracy_RF4=predict_probo_class(month_alpha_factor_series,RF,12,month_list,0.4,-1)
Mfactor0_RF3,Mfactor1_RF3,Mtest_accuracy_RF3,Mtrain_accuracy_RF3=predict_probo_class(month_alpha_factor_series,RF,12,month_list,0.3,-1)

In [ ]:
#周频数据(二分类，按照30%和40%划分标签值)
Wfactor0_lgb3,Wfactor1_lgb3,Wtest_accuracy_lgb3,Wtrain_accuracy_lgb3=predict_probo_class(week_alpha_factor_series,model_lgb,48,week_list,0.4,-1)
Wfactor0_lgb4,Wfactor1_lgb4,Wtest_accuracy_lgb4,Wtrain_accuracy_lgb4=predict_probo_class(week_alpha_factor_series,model_lgb,48,week_list,0.3,-1)

Wfactor0_xgb4,Wfactor1_xgb4,Wtest_accuracy_xgb4,Wtrain_accuracy_xgb4=predict_probo_class(week_alpha_factor_series,model_xgb,12,week_list,0.4,-1)
Wfactor0_xgb3,Wfactor1_xgb3,Wtest_accuracy_xgb3,Wtrain_accuracy_xgb3=predict_probo_class(week_alpha_factor_series,model_xgb,12,week_list,0.3,-1)
                                                                                         
Wfactor0_GBoost4,Wfactor1_GBoost4,Wtest_accuracy_GBoost4,Wtrain_accuracy_GBoost4=predict_probo_class(week_alpha_factor_series,GBoost,12,week_list,0.4,-1)
Wfactor0_GBoost3,Wfactor1_GBoost3,Wtest_accuracy_GBoost3,Wtrain_accuracy_GBoost3=predict_probo_class(week_alpha_factor_series,GBoost,12,week_list,0.3,-1)

Wfactor0_RF4,Wfactor1_RF4,Wtest_accuracy_RF4,Wtrain_accuracy_RF4=predict_probo_class(week_alpha_factor_series,RF,12,week_list,0.4,-1)
Wfactor0_RF3,Wfactor1_RF3,Wtest_accuracy_RF3,Wtrain_accuracy_RF3=predict_probo_class(week_alpha_factor_series,RF,12,week_list,0.3,-1)

## 预测准确率分析

In [ ]:
accu_score(Wtest_accuracy_lgb3,"score30_train")
accu_score(Wtrain_accuracy_lgb3,"score30_train")
accu_score(Wtest_accuracy_lgb4,"score40_train")
accu_score(Wtrain_accuracy_lgb4,"score40_train")

accu_score(Mtest_accuracy_xgb4,,"score40_train")
accu_score(Mtrain_accuracy_xgb4,"score40_train")
accu_score(Mtest_accuracy_xgb3,,"score30_train")
accu_score(Mtrain_accuracy_xgb3,"score30_train")

## 因子分析

### 因子保存

In [ ]:
import pickle
Wfactor1_lgb3.to_pickle("./factor_test_data/LightGBM_allfactor2/Wfactor1_lgb3")
Wfactor1_lgb4.to_pickle("./factor_test_data/LightGBM_allfactor2/Wfactor1_lgb4")

### 因子测试

In [ ]:
# #因子测试1--月频数据
Mfactor1_lgb3_obj=factor_analyse("test1",Mfactor1_lgb3)
Mfactor1_lgb4_obj=factor_analyse("test1",Mfactor1_lgb4)

Mfactor1_xgb3_obj=factor_analyse("test1",Mfactor1_xgb3)
Mfactor1_xgb4_obj=factor_analyse("test1",Mfactor1_xgb4)

Mfactor1_RF3_obj=factor_analyse("test1",Mfactor1_RF3)
Mfactor1_RF4_obj=factor_analyse("test1",Mfactor1_RF4)

Mfactor1_GBoost3_obj=factor_analyse("test1",Mfactor1_GBoost3)
Mfactor1_GBoost4_obj=factor_analyse("test1",Mfactor1_GBoost4)

# #因子测试2--月频数据
Msamew_lgb3_pre,Msamew_lgb3_unpre,Munw_lgb3_pre,Munw_lgb3_unpre=factor_stock_choose(Mfactor1_lgb3,100)
Msamew_lgb4_pre,Msamew_lgb4_unpre,Munw_lgb4_pre,Munw_lgb4_unpre=factor_stock_choose(Mfactor1_lgb4,100)

Msamew_xgb3_pre,Msamew_xgb3_unpre,Munw_xgb3_pre,Munw_xg34_unpre=factor_stock_choose(Mfactor1_xgb3,100)
Msamew_xgb4_pre,Msamew_xgb4_unpre,Munw_xgb4_pre,Munw_xgb4_unpre=factor_stock_choose(Mfactor1_xgb4,100)

Msamew_GBoost4_pre,Msamew_GBoost4_unpre,Munw_GBoost4_pre,Munw_GBoost4_unpre=factor_stock_choose(Mfactor1_GBoost4,100)
Msamew_GBoost3_pre,Msamew_GBoost3_unpre,Munw_GBoost3_pre,Munw_GBoost3_unpre=factor_stock_choose(Mfactor1_GBoost3,100)

Msamew_RF3_pre,Msamew_RF3_unpre,Munw_GRF3_pre,Munw_RF3_unpre=factor_stock_choose(Mfactor1_RF3,100)
Msamew_RF4_pre,Msamew_RF4_unpre,Munw_RF4_pre,Munw_RF4_unpre=factor_stock_choose(Mfactor1_RF4,100)

In [ ]:
# #因子测试1--周频数据
Wfactor1_lgb3_obj=factor_analyse("test1",Wfactor1_lgb3)
Wfactor1_lgb4_obj=factor_analyse("test1",Wfactor1_lgb4)

Wfactor1_xgb3_obj=factor_analyse("test1",Wfactor1_xgb3)
Wfactor1_xgb4_obj=factor_analyse("test1",Wfactor1_xgb4)

Wfactor1_RF3_obj=factor_analyse("test1",Wfactor1_RF3)
Wfactor1_RF4_obj=factor_analyse("test1",Wfactor1_RF4)

Wfactor1_GBoost3_obj=factor_analyse("test1",Wfactor1_GBoost3)
Wfactor1_GBoost4_obj=factor_analyse("test1",Wfactor1_GBoost4)

# #因子测试2--周频数据
Wsamew_lgb3_pre,Wsamew_lgb3_unpre,Wunw_lgb3_pre,Wunw_lgb3_unpre=factor_stock_choose(Wfactor1_lgb3,100)
Wsamew_lgb4_pre,Wsamew_lgb4_unpre,Wunw_lgb4_pre,Wunw_lgb4_unpre=factor_stock_choose(Wfactor1_lgb4,100)

Wsamew_xgb3_pre,Wsamew_xgb3_unpre,Wunw_xgb3_pre,Wunw_xg34_unpre=factor_stock_choose(Wfactor1_xgb3,100)
Wsamew_xgb4_pre,Wsamew_xgb4_unpre,Wunw_xgb4_pre,Wunw_xgb4_unpre=factor_stock_choose(Wfactor1_xgb4,100)

Wsamew_GBoost4_pre,Wsamew_GBoost4_unpre,Wunw_GBoost4_pre,Wunw_GBoost4_unpre=factor_stock_choose(Wfactor1_GBoost4,100)
Wsamew_GBoost3_pre,Wsamew_GBoost3_unpre,Wunw_GBoost3_pre,Wunw_GBoost3_unpre=factor_stock_choose(Wfactor1_GBoost3,100)

Wsamew_RF3_pre,Wsamew_RF3_unpre,Wunw_GRF3_pre,Wunw_RF3_unpre=factor_stock_choose(Wfactor1_RF3,100)
Wsamew_RF4_pre,Wsamew_RF4_unpre,Wunw_RF4_pre,Wunw_RF4_unpre=factor_stock_choose(Wfactor1_RF4,100)

### 显示分析图

In [ ]:
show2(Wfactor1_lgb3_obj)

## 函数部分

#### 分类预测

In [ ]:
def predict_probo_class(data,model,loop,date_list,T,x):
    train,test=data_input_class(data,date_list,T,x)  
    predict_value=[]
    test_accuracy=[]
    train_accuracy=[]
    start_clock  = time.time()
    for m in range(len(date_list)-loop):
        X_train, X_test, Y_train, Y_test,Xx_test= splitdata(train,test,m+loop-1,m,m+loop-1,m+loop)
        model.fit(X_train,Y_train)
        predict=model.predict_proba(X_test)
        predict=pd.DataFrame(predict)
        score_test=model.score(X_test,Y_test)
        score_train=model.score(X_train,Y_train)
        predict['stock']=Xx_test[:,-2]
        predict['date']=Xx_test[:,-1] 
        predict_value.append(predict)
        test_accuracy.append(score_test)
        train_accuracy.append(score_train)
    end_clock = time.time()
    print('cpu cost time: ', end_clock - start_clock)
    factor0_df,factor1_df=changeindex2(predict_value,loop,date_list)
    return  factor0_df,factor1_df,test_accuracy,train_accuracy  

#### 分类模型(GBoost，RF，XGB，LightGBM)

In [ ]:
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

GBoost = GradientBoostingClassifier(n_estimators=3000, learning_rate=0.05,max_depth=5)
RF = RandomForestClassifier(max_depth=5)
model_xgb = xgb.XGBClassifier(learning_rate=0.05, max_depth=5, )
model_lgb = lgb.LGBMClassifier(num_leaves=50,learning_rate=0.01)

#### 数据构造

In [ ]:
#标签值--分类
def rise_fall_class(trade_date_m_series,T):
    zz_df = dao.get_index_component_data('ZZ')
    stock_price_df = dao.get_security_info('stock_price_info')[trade_date_m_series]
    stock_close_df = stock_price_df.xs('close',level=1)[trade_date_m_series]
    trade_status_df = stock_price_df.xs('trade_status', level=1)[trade_date_m_series]
    pause_df = trade_status_df.copy()
    pause_df[pause_df == 1] = np.nan
    pause_df[pause_df==0] = 1

    stock_chg_df = stock_close_df.pct_change(axis=1)
    stock_return_df = stock_chg_df * pause_df  * zz_df
    stock_return_df = stock_return_df.shift(-1, axis=1)

    is_rise_df = stock_return_df.copy()
    rise_quatile_percent = T
    rise_quan30_series = is_rise_df.quantile(rise_quatile_percent)
    rise_quan70_series = is_rise_df.quantile(1.0 - rise_quatile_percent)

    is_rise_df[is_rise_df>rise_quan70_series] = 999
    is_rise_df[is_rise_df<rise_quan30_series] = -999
    is_rise_df[is_rise_df.abs() != 999] = np.nan

    is_rise_df.replace(to_replace=[-999, 999], value=[-1, 1], inplace=True)
    return is_rise_df

#划分测试集和训练集
def concat_data(data,date_list,is_rise_df):
    factor_class_series = data.map(lambda x: x.loc[:,date_list[0]:date_list[-1]])
    data_df = pd.DataFrame()
    factor_name_list = factor_class_series.index.tolist()
    data_dict = {}
    for trade_date in tqdm(date_list[:]):
        data_section_series = factor_class_series.map(lambda x: x[trade_date] if trade_date in x.columns else None)
        data_section_df = pd.DataFrame(data_section_series.to_dict())
        data_section_df =data_section_df.reindex(columns=factor_name_list)
        data_section_df['rise_fall'] = is_rise_df[trade_date]
        data_dict[trade_date] = data_section_df
        data_section_df['date'] = trade_date
    return data_dict
    
def data_train_test(data_pct,x):
    data_pct['rise_fall']=data_pct['rise_fall'].fillna(x)    #注意fillna填充！！！(x=-1是标志值二分类，分为“-1,1”两类)
#     data_pct['rise_fall']=data_pct['rise_fall'].fillna(0)   #x=0是标志值三分类，分为“-1,0，1”两类
    data_pct_test=data_pct      #包含0，-1,1的三种分类的全部数据预测集
    data_pct_test=data_pct[data_pct['trade_status']==0]        #选择正常股票状态的数据
#     data_pct_test=data_pct_test.dropna()    #删除空值
    data_pct_train=data_pct[~data_pct['rise_fall'].isin([0])]        #不包含0的训练集
    data_pct_train=data_pct_train[data_pct_train['trade_status']==0]       #选择正常股票状态的数据
#     data_pct_dropna_train=data_pct_train.dropna()
    return data_pct_train,data_pct_test

def data_input_class(data,date_list,T,x):
    is_rise_df=rise_fall_class(date_list,T)    
    data_dict=concat_data(data,date_list,is_rise_df)
    data_df = pd.concat([data_dict[frame] for frame in data_dict.keys()])
    train1,test1=data_train_test(data_df,x) 
    return train1,test1

# 数据标准化，可以处理空值
def standard(X_train,X_test):
    X_train_scaled =1.0 * (X_train - X_train.mean()) / X_train.std()  # 数据标准化
    X_test_scaled =1.0 * (X_test - X_test.mean()) / X_test.std()  # 数据标准化
    return  X_train_scaled,X_test_scaled 

def countsum(data):
    a=data.reset_index()
    a.rename(columns=lambda x:x.replace('index','stock'), inplace=True) 
    resultdata=(a['stock'].groupby(a['date'])).describe()
    resultdata['sum']=resultdata['count'].cumsum()
    return resultdata,a

#两分类划分，划分训练集data，测试集alldata（训练集的类别只有（0,1），测试集包含所有类别（0,1，-1））
def splitdata(data,alldata,i,j,x,y):
    resultdata,a=countsum(data)
    resultalldata,b=countsum(alldata)
    i=resultdata['sum'][i]
    j=resultdata['sum'][j]
    x=resultalldata['sum'][x]
    y=resultalldata['sum'][y]

    newname=fill_alpha_factor_series.index.tolist()    
    newname.append('stock')
    newname.append('date')

    X_train=np.array(a[newname][j:i])
    Y_train=np.array(a['rise_fall'][j:i])
    #第x个月，测试集
    X_test=np.array(b[newname][x:y])
    Y_test=np.array(b['rise_fall'][x:y])
    X_train_scaled,X_test_scaled=X_train[:,:-3],X_test[:,:-3]

    return X_train_scaled,X_test_scaled,Y_train,Y_test,X_test

#### 缺失值填充

In [ ]:
#按照行业均值进行填充
def fillna_ind_mean(data,date_list):
    industry_class_df=dao.get_stock_industry_data('CS')               #股票行业信息----周频信息
    industry_class_m_df=industry_class_df.loc[:,date_list]    #股票行业信息----月频信息
    #按照行进行填充，下一个值
    industry_class_m_df=industry_class_m_df.fillna(method='bfill',axis=1)
    industry_class_m_df=industry_class_m_df.fillna('未知')  

    for i in range(1,len(date_list)):
        resl_series = pd.Series() 
        industry_series=industry_class_m_df.iloc[:,i]
        group_data = industry_series.index.groupby(industry_series.values) 
        industry_list = list(group_data.keys())   
        data_series=data.iloc[:,i]
        for industry_name in industry_list:      
            industry_temp = data_series.loc[group_data[industry_name]]
            industry_temp = industry_temp.fillna(industry_temp.mean())         
            resl_series = resl_series.append(industry_temp)
        stock_list = list(set(data_series.index) - set(industry_series.dropna().index))
        resl_series = resl_series.append(data_series.loc[stock_list])
        data.iloc[:,i]=resl_series
    return data

#只做均值填充
def data_fillna_mean(df,date_list):
    df=fillna_ind_mean(df,date_list)
    return df

#空值填充，行业均值填充
def nonefill(data,date_list):
    tempdata=data
    for i in tempdata:
        i=data_fillna_mean(i,date_list)
    return tempdata

#### 预测准确率分析

In [ ]:
#预测准确率分析
def accu_score(score,name):
    plt.axhline(0.5,color='red')
    plt.plot(score)
    plt.text(score[1],score[1],name,fontdict={'size':'16','color':'black'})
    plt.title("Accuracy on TestData", fontsize=15)

#### 因子数据合成

In [ ]:
import itertools

def change(data,n,m,M,date_list):
    date=date_list[M:]
    factor_df=pd.DataFrame(columns=date)
    factor_df['stock']=list(fill_alpha_factor_series[100].index)
    for i,t in itertools.zip_longest(data,date):
        temp=factor_df[['stock']]
        temp[t]=np.nan
        u=i.iloc[:,[n,m]]
        u.columns=[t,'stock']
        factor_Crash=pd.concat([u,temp],join='inner',ignore_index=True)
        factor_Crash.sort_values(t,inplace=True)
        factor_Crash.drop_duplicates(['stock'],inplace=True)
        factor_Crash.sort_values('stock',inplace=True)
        factor_Crash.reset_index(inplace=True)
        factor_df[t]= factor_Crash[t]
    factorF_df=factor_df.set_index(['stock'])
    return factorF_df

def changeindex2(data,M,date_list):
    factor0_df=change(data,0,2,M,date_list)
    factor1_df=change(data,1,2,M,date_list)
    return  factor0_df,factor1_df

#### 因子显著度T检验

In [ ]:
import util.factortool as ftool
def factor_test_T(factor_list,factor_name): 
    risk_test=pd.DataFrame()
    for i ,n in itertools.zip_longest(factor_list,factor_name):
        risk_test[n]=ftool.factor_risk_test_tvalue(i)
    return  risk_test

#### 因子测试1

In [ ]:
import alphafactors.factorprepro_class as fp
import alphafactors.factoranalyse as fa

#因子处理(分成两种方向)
def factor_analyse(name,factor):  # 0-positive , 1-negetive
    factor_prepro_obj = fp.FactorPrePro(factor_name=name, factor_data_df=factor, universe='ZZ', neutral_list=None)
    factor_prepro_obj.run_process(start_date=max(factor.columns[0], dt.datetime(2007,1,5)), end_date=factor.columns[-1])
    df = factor_prepro_obj.factor_pro_df
    factor_test_obj = fa.FactorAnalyse(factor_name=name, factor_data_df=df, factor_dr=0)   # 0-positive , 1-negetive
    factor_test_obj.run_analyse_new(start_date=dt.datetime(2009,1,23), universe='ZZ')
    return factor_test_obj

#因子测试画图显示
def show1(factor_test_obj):
    factor_test_obj.net_value_df.iloc[:,-3:].plot(figsize=(20,10))
def show2(factor_test_obj):
    factor_test_obj.factor_perform_df
    return  factor_test_obj.factor_perform_df
def show3(factor_test_obj):
    factor_test_obj.factor_para_df
    return  factor_test_obj.factor_para_df
def show4(factor_test_obj):
    factor_test_obj.port_perform_df
    return     factor_test_obj.port_perform_df
def show5(factor_test_obj):
    factor_test_obj.port_perform_df['annual_return'].plot(kind='bar')
    return factor_test_obj.port_perform_df['annual_return'].plot(kind='bar')   
def show6(factor_test_obj):
    factor_test_obj.factor_index_df['IC值'].plot(kind='bar', figsize=(20,10), color='blue')
    return  factor_test_obj.factor_index_df['IC值'].plot(kind='bar', figsize=(20,10), color='blue')

#### 因子测试2

In [ ]:
import util.evalstat as evl
                                     
def factor_test_pre(factor):         #因子中性化预处理
    factor_prepro_obj = fp.FactorPrePro(factor_name='factor_test', factor_data_df=factor, universe='ZZ', neutral_list=None)
    factor_prepro_obj.run_process(start_date=max(factor.columns[0], dt.datetime(2007,1,5)), end_date=factor.columns[-1])
    df = factor_prepro_obj.factor_pro_df
    return df

def factor_test(stock_weighted_series):
    perform_obj=evl.PortPerform(port_series=stock_weighted_series,ret_type='open',fee=0.0035)
    perform_obj.run()
    return perform_obj

def show01(perform_obj):
    perform_obj.net_value_plot()        
def show02(perform_obj):
    perform_obj.get_strategy_perform() 
    return perform_obj.get_strategy_perform()      
def show03(perform_obj):
    perform_obj.get_avg_turnover()  
    return perform_obj.get_avg_turnover()      
def show04(perform_obj):
    perform_obj.get_annual_perform()  
    return  perform_obj.get_annual_perform()  

#### 投资组合构建（2种：一种等权，一种不等权）

In [ ]:
def stock_choice(data,num):                 #直接挑选概率值前100支股票，等权
    stock_series= pd.Series()
    for i in data.columns:
        stock_series.loc[i]=pd.Series(index=[data[i].sort_values(ascending=False).head(num).index],data=1/num)
    stock_choice_obj=factor_test(stock_series)
    
    return stock_choice_obj

def stock_bench_ind(data,num):   #行业中性，基准权重后挑选100支股票
    stock_series= pd.Series()
    for i in data.columns:
       set_date=i
       stock_series[i]=get_industry_stock(data[i], set_date, stock_num=num)
    stock_choice_obj =factor_test(stock_series)

    return stock_choice_obj

def factor_stock_choose(factor,num):
    factor_obj=factor_test_pre(factor)   #做因子预处理
    #等权选num支
    samew_pre=stock_choice(factor_obj,num)
    samew_unpre=stock_choice(factor,num)   #不做因子预处理，等权直接选100支
    #不等权选num支
    unw_pre=stock_bench_ind(factor_obj,num)
    unw_unpre=stock_bench_ind(factor,num)
    return samew_pre,samew_unpre,unw_pre,unw_unpre

#### 基准行业权重

In [ ]:
stock_industry_df = dao.get_security_info('stock_industry_CS')
stock_industry_list = dso.get_industry_classify('CS')
stock_pool_df = dao.get_index_component_data('DEF')

def get_bench_ind_weight(set_date, bench_code='ZZ500'):
    industry_series = optool.get_series_from_df(data_df=stock_industry_df, set_date=set_date, axis=1).dropna()
    group_data = industry_series.index.groupby(industry_series)
    # 基准行业权重
    bench_component_df = dao.get_index_component_data(bench_code)
    bench_series = pd.Series(bench_component_df[set_date].set_index('code')['weight'])
    bench_series = bench_series / bench_series.sum()
    bench_ind_weight_series = pd.Series(index=stock_industry_list)
    for industry_name in stock_industry_list:
        ind_stock_list = group_data[industry_name]
        temp_series = bench_series.copy()
        bench_ind_weight_series.loc[industry_name] = temp_series.reindex(ind_stock_list).sum()
    bench_ind_weight_series.fillna(0, inplace=True)
    return bench_ind_weight_series


def get_industry_stock(stock_factor_series, set_date, stock_num=100):
    industry_series = optool.get_series_from_df(data_df=stock_industry_df, set_date=set_date, axis=1).dropna()
    stock_series = stock_pool_df[set_date]
    stock_list = industry_series.index.intersection(stock_series.dropna().index).tolist()
    industry_series = industry_series.loc[stock_list]

    group_data = industry_series.index.groupby(industry_series)

    # 基准行业权重
    bench_ind_weight_series = get_bench_ind_weight(set_date=set_date)
    bench_ind_num_series = round(bench_ind_weight_series * stock_num).astype(int)

    port_series = pd.Series()
    # 得到行业中性组合
    for industry_name in stock_industry_list[:]:
        if bench_ind_weight_series[industry_name] <= 0.0:
            continue
        ind_stock_list = group_data[industry_name]
        ind_stock_series = pd.Series(stock_factor_series.loc[ind_stock_list]).reindex(ind_stock_list).sort_values(
            ascending=False)

        ind_stock_num = bench_ind_num_series[industry_name]
        if ind_stock_num < 1:
            ind_stock_num += 1

        if ind_stock_series.shape[0] >= ind_stock_num:
            xx = ind_stock_series.head(ind_stock_num)
            temp_series = xx / xx.sum() * bench_ind_weight_series[industry_name]
        else:
            temp_series = ind_stock_series / ind_stock_series.sum() * bench_ind_weight_series[industry_name]

            temp_series = pd.Series(index=ind_stock_series.index[:ind_stock_num], data=1.0 / ind_stock_num) * \
                          bench_ind_weight_series[industry_name]
        port_series = port_series.append(temp_series)
    return port_series